In [1]:
import numpy as np
import cv2
import time

import os
import sys
import glob
import dlib
from skimage import io

In [2]:
detector = dlib.simple_object_detector("airplane.svm")

In [3]:
cap = cv2.VideoCapture('airplane.mp4')
tmp = []
tmp_img = []
img = []
cv2.namedWindow('video')
cv2.namedWindow('tmp')
tl = (0, 0)
br = (0, 0)
rect = (0,0,1,1)
rectangle = False
rect_over = False 

In [4]:
def match(image, template):
    # resize images
    sh = image.shape
    image = cv2.resize(image, (0,0), fx=0.5, fy=0.5) 
    template = cv2.resize(template, (0,0), fx=0.5, fy=0.5) 
    # Convert to grayscale
    imageGray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    templateGray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

    #t = time.clock() 
    # Find template
    result = cv2.matchTemplate(imageGray,templateGray, cv2.TM_CCOEFF)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
    top_left = max_loc
    h,w = templateGray.shape
    bottom_right = (top_left[0] + w, top_left[1] + h)
    
    return (top_left[0]*2, top_left[1]*2), (bottom_right[0]*2, bottom_right[1]*2) 

In [5]:
def control(contour, shape):
    tr = 10
    for cnt in contour[0]:
        if cnt[0] < tr or cnt[0] > shape[1]-tr or cnt[1] < tr or cnt[1] > shape[0]-tr:
            return False
    return True 

In [6]:
def getRect(contour, shape):
    tr = 10
    x1,y1,x2,y2 = (100000,1000000,0,0)
    for cnt in contour[0]:
        for c in cnt:
            if x1 > c[0]:
                x1 = c[0]
            if x2 < c[0]:
                x2 = c[0]
            if y1 > c[1]:
                y1 = c[1]
            if y2 < c[1]:
                y2 = c[1]
    if x1 < tr or x2 > shape[0]-tr or y1 < tr or y2 > shape[1]-tr:
        return None
    return (x1, y1, x2-x1, y2-y1)

In [7]:
def draw(img):
    area = img.shape[0]*img.shape[1]/10
    tmp = np.copy(img)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    cv2.imshow('gray', gray)
    median = cv2.medianBlur(gray, 5)
    cv2.imshow('median', median)
    contours = []
    #ret, thresh = cv2.threshold(median, th, 255, 0)
    for bs  in range (21, 222, 20):
        thresh = cv2.adaptiveThreshold(median,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,bs,2)
        cv2.imshow('thresh'+str(bs), thresh)
        #edges = cv2.Canny(thresh, 100, 200)
        #cv2.imshow('canny'+str(bs), edges)
        _,conts, hierarchy = cv2.findContours(thresh, 1, 2)# cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        contours += conts
    cont = []
    for cnt in contours:
        if cv2.contourArea(cnt) > area:
            appr = cv2.approxPolyDP(cnt, 0.003*cv2.arcLength(cnt, True), True)
            if control(appr, gray.shape) is True:
                cont.append(appr)
    if cont != []:
        coords = getRect(cont, gray.shape)
        if coords is not None:
            x,y,w,h = coords
            cv2.rectangle(tmp, (x,y), (x+w,y+h), (0,255,0), 4)
            cv2.imshow('cnts', tmp)    
            return (x,y,w,h)
    cv2.imshow('cnts', tmp)    
    return None


q = 0

In [8]:

global tmp,rect
count = 0
while cap.isOpened():
    q = q+1
    ret, img1 = cap.read()
    if ret:
        cv2.imshow('video',img1)
        cv2.waitKey(1)
        img2 = np.copy(img1)  
        while(rect_over == False):
            if ret:
                dets = detector(img1)
                print("Processing file: {}".format(img1))
                print("Number of airplanes detected: {}".format(len(dets)))
                cv2.waitKey(1)
                for k, d in enumerate(dets):
                    print("Detection {}: Left: {} Top: {} Right: {} Bottom: {}".format(
                        k, d.left(), d.top(), d.right(), d.bottom()))
                    tl = (d.left(),d.top())
                    w = abs(d.right()-d.left())
                    h = abs(d.top()-d.bottom()) 
                    br = (d.right(),d.bottom())
                    tmp = img1[tl[0]:tl[0]+h  , tl[1]:tl[1]+w]
                    height, width, channels = tmp.shape
                    print(tmp.shape)
                    if height == 0 or width == 0:
                        break
                    print(tmp.shape)
                    print(height)
                    cv2.rectangle(img1, (d.left(), d.top()), (d.right(), d.bottom()), (0,0,255), 5)
                    rect_over = True
                ret,img1 = cap.read()
        if rect_over:
            coords = draw(tmp)
            if coords is not None and q>=6:
                q = 0
                x,y,w,h = coords
                x1, y1 = max(0, tl[0] + (x - (br[0] - tl[0] - w - x))/2), max(0, tl[1] + (y - (br[1] - tl[1] - h - y))/2)
                tl, br = (x1, y1), (x1 + br[0]-tl[0], y1 + br[1]-tl[1])
            else:
                tl, br = match(img2, tmp)
            #tmp = img[tl[1]:br[1], tl[0]:br[0]]
            cv2.imshow('tmp', tmp)
        cv2.rectangle(img2,tl,br,(0,0,255),5)
        cv2.imshow('video', img2)
        cv2.waitKey(50)
        if cv2.waitKey(20) &  0xFF == ord('q'):
            break
    else:
        break
cap.release()
cv2.destroyAllWindows()


Processing file: [[[126 114 121]
  [126 114 121]
  [126 114 121]
  ...
  [121 107 105]
  [121 107 105]
  [121 107 105]]

 [[126 114 121]
  [126 114 121]
  [126 114 121]
  ...
  [121 107 105]
  [121 107 105]
  [121 107 105]]

 [[126 114 121]
  [126 114 121]
  [126 114 121]
  ...
  [121 107 105]
  [121 107 105]
  [121 107 105]]

 ...

 [[ 91  78  82]
  [ 91  78  82]
  [ 91  78  82]
  ...
  [141 135 143]
  [141 135 143]
  [141 135 143]]

 [[ 91  78  82]
  [ 91  78  82]
  [ 91  78  82]
  ...
  [141 135 143]
  [141 135 143]
  [141 135 143]]

 [[ 91  78  82]
  [ 91  78  82]
  [ 91  78  82]
  ...
  [141 135 143]
  [141 135 143]
  [141 135 143]]]
Number of airplanes detected: 1
Detection 0: Left: 286 Top: 286 Right: 881 Bottom: 584
(298, 595, 3)
(298, 595, 3)
298
